In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install streamlit>=1.20.0 tensorflow>=2.10.0 transformers>=4.26.0 emoji>=2.2.0 pyngrok>=5.1.0

In [3]:


#  Create the Streamlit app file
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification
import re
import emoji

# Set page configuration
st.set_page_config(
    page_title="Multilingual Hate Speech Detector",
    page_icon="🛡️",
    layout="wide"
)

@st.cache_resource
def load_model_and_tokenizer():
    """Load the model and tokenizer with caching to improve performance"""
    # Path to your saved model in Google Drive
    model_path = "/content/drive/MyDrive/ProjetNLP/best_multilingual_hate_speech_model.pt"

    # Use RoBERTa tokenizer instead of BERT
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

    try:
        # Initialize a RoBERTa model
        model = RobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

        # Load the state dict from the .pt file
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        model.eval()  # Set to evaluation mode
        st.success("Model loaded successfully!")
    except Exception as e:
        st.error(f"Error loading model: {e}")
        st.error("Make sure your model is saved at '/content/drive/MyDrive/ProjetNLP/best_multilingual_hate_speech_model.pt'")

        # Additional information for debugging
        st.error("Your model appears to be a RoBERTa model. Make sure you're using the correct model architecture.")
        raise e

    return model, tokenizer

def preprocess_text(text):
    """Clean the input text similar to training preprocessing"""
    # Remove mentions
    text = re.sub(r"@[\d\w_]+\s?", "", text)
    # Remove URLs
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", "", text)
    # Remove Twitter image URLs
    text = re.sub(r"pic.twitter.com/[\w\d]+", "", text)
    # Convert emojis to text
    text = emoji.demojize(text)
    return text

def predict_hate_speech(text, model, tokenizer):
    """Make prediction on whether input text contains hate speech"""
    # Preprocess the text
    cleaned_text = preprocess_text(text)

    # Tokenize
    inputs = tokenizer(
        cleaned_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"  # PyTorch tensors
    )

    # Make prediction
    with torch.no_grad():  # No gradient computation needed for inference
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1).numpy()[0]

    # Get prediction (0 = not hate speech, 1 = hate speech)
    prediction = int(probabilities[1] > 0.5)  # Use threshold
    confidence = float(probabilities[prediction])

    return {
        "is_hate_speech": bool(prediction),
        "confidence": confidence,
        "cleaned_text": cleaned_text,
        "probabilities": {
            "not_hate_speech": float(probabilities[0]),
            "hate_speech": float(probabilities[1])
        }
    }

def get_token_contributions(text, model, tokenizer):
    """Analyze which words contribute most to the hate speech classification"""
    cleaned_text = preprocess_text(text)
    tokens = tokenizer.tokenize(cleaned_text)

    # Get baseline prediction for the full text
    baseline_result = predict_hate_speech(cleaned_text, model, tokenizer)
    baseline_prediction = baseline_result["is_hate_speech"]

    # If there are too many tokens, focus on a subset
    max_tokens_to_analyze = min(len(tokens), 25)  # Reduced for Colab performance
    tokens_to_analyze = tokens[:max_tokens_to_analyze]

    token_impacts = []

    # For each token, measure the impact of removing it
    for i, token in enumerate(tokens_to_analyze):
        if token.startswith('Ġ') or token in ['<s>', '</s>', '.', ',', '!', '?']:
            continue  # Skip special tokens in RoBERTa

        # Create a version of the text without this token
        modified_tokens = tokens.copy()
        modified_tokens[i] = "<mask>"  # RoBERTa uses <mask> instead of [MASK]
        modified_text = tokenizer.convert_tokens_to_string(modified_tokens)

        # Predict on the modified text
        result = predict_hate_speech(modified_text, model, tokenizer)

        # Calculate the impact of this token
        impact = 0
        if baseline_prediction:  # If original text is classified as hate speech
            # Impact is positive if removing the token reduces the hate probability
            impact = baseline_result["confidence"] - result["confidence"]
        else:  # If original text is not classified as hate speech
            # Impact is positive if removing the token increases the non-hate probability
            impact = result["confidence"] - baseline_result["confidence"]

        token_impacts.append({
            "token": token,
            "impact": impact
        })

    # Sort by absolute impact
    token_impacts.sort(key=lambda x: abs(x["impact"]), reverse=True)

    return token_impacts[:5]  # Return top 5 impactful tokens

def main():
    st.title("Multilingual Hate Speech Detector")
    st.markdown("""
    This application detects hate speech in multiple languages using a fine-tuned XLM-RoBERTa model.
    Enter text in any language and the model will analyze it for hate speech content.
    """)

    # Load model and tokenizer
    with st.spinner("Loading model... This may take a moment."):
        try:
            model, tokenizer = load_model_and_tokenizer()
        except Exception as e:
            st.error("Failed to load the model. Check the error message above.")
            return

    # Text input
    text_input = st.text_area("Enter text to analyze:", height=150)

    analyze_button = st.button("Analyze Text")

    if analyze_button:
        if not text_input:
            st.warning("Please enter some text to analyze.")
        else:
            with st.spinner("Analyzing text..."):
                try:
                    # Make prediction
                    result = predict_hate_speech(text_input, model, tokenizer)

                    # Display results
                    col1, col2 = st.columns(2)

                    with col1:
                        if result["is_hate_speech"]:
                            st.error(f"⚠️ Hate speech detected with {result['probabilities']['hate_speech']*100:.2f}% confidence")
                        else:
                            st.success(f"✅ No hate speech detected with {result['probabilities']['not_hate_speech']*100:.2f}% confidence")

                        # Display probability bars
                        st.markdown("### Probability Analysis")
                        st.markdown("**Not Hate Speech:**")
                        st.progress(float(result['probabilities']['not_hate_speech']))
                        st.markdown(f"{result['probabilities']['not_hate_speech']*100:.2f}%")

                        st.markdown("**Hate Speech:**")
                        st.progress(float(result['probabilities']['hate_speech']))
                        st.markdown(f"{result['probabilities']['hate_speech']*100:.2f}%")

                    # Get and display token contributions
                    with col2:
                        st.subheader("Word Impact Analysis")
                        try:
                            token_impacts = get_token_contributions(text_input, model, tokenizer)

                            if token_impacts:
                                st.markdown("These words had the most impact on the classification:")
                                for item in token_impacts:
                                    impact_direction = "increases" if item["impact"] > 0 else "decreases"
                                    st.markdown(f"- **{item['token'].replace('Ġ', '')}**: {impact_direction} hate speech probability by {abs(item['impact']*100):.2f}%")
                            else:
                                st.info("No significant word impacts found")
                        except Exception as word_error:
                            st.warning(f"Could not perform word impact analysis: {word_error}")

                    # Display the processed text
                    st.subheader("Processed Text")
                    st.text(result["cleaned_text"])

                except Exception as analysis_error:
                    st.error(f"Error analyzing text: {analysis_error}")

if __name__ == "__main__":
    main()



Writing app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

35.201.151.4


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.201.151.4:8501

⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://lucky-horses-throw.loca.lt
2025-03-04 07:47:59.338438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741074479.368611    4967 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741074479.377396    4967 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 07:47:59.406675: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized 

In [ ]:
import torch
model = torch.load('/content/drive/MyDrive/ProjetNLP/best_multilingual_hate_speech_model.pt')
print(type(model))
# If it's a dict, print the keys
if isinstance(model, dict):
    print(list(model.keys()))

<ipython-input-20-37acd57a2bf7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/ProjetNLP/best_multilingual_hate_speech_model.pt'

<class 'collections.OrderedDict'>
['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediate.dense.bias', 'roberta.encoder.layer.0.output.dense.weight', 'roberta.encoder.layer.0.output.dense.bia

Overwriting app.py
